# Features acteurs oscar

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Aller on taff

In [2]:
acteurs_df = pd.read_csv('../raw_data/cat_acteur.csv')


In [3]:
acteurs_df

,Unnamed: 0,acteur_name,title,year,month,budget,income,ratio
0,0,Werner Krauss,Il gabinetto del dottor Caligari,1920,2,1.800000e+04,8811,0.489500
1,1,Conrad Veidt,Il gabinetto del dottor Caligari,1920,2,1.800000e+04,8811,0.489500
2,2,Friedrich Feher,Il gabinetto del dottor Caligari,1920,2,1.800000e+04,8811,0.489500
3,3,Amund Rydland,Markens grøde,1921,12,2.926274e+04,4272,0.145988
4,4,Karen Poulsen,Markens grøde,1921,12,2.926274e+04,4272,0.145988
...,...,...,...,...,...,...,...,...
37096,37096,Gopika Anil,Munthiri Monchan,2019,12,3.975771e+05,2568,0.006459
37097,37097,Ramana,Kaithi,2019,10,3.180617e+06,524061,0.164767
37098,37098,Nandu Anand,Ottam,2019,3,5.301028e+04,4791,0.090379
37099,37099,Roshan Ullas,Ottam,2019,3,5.301028e+04,4791,0.090379


## On définit le ratio de chaque films

In [4]:
acteurs_df['ratio'] = acteurs_df['income']/acteurs_df['budget']

### Détermination des catégories des acteurs

In [5]:
acteurs_df_cat = acteurs_df.loc[(acteurs_df['ratio']>=5) & (acteurs_df['budget'] >= 100_000_000)]

In [6]:
len(acteurs_df_cat['acteur_name'].unique())

159

In [7]:
acteurs_df_cat[['acteur_name','budget']].groupby(by='acteur_name').sum().sort_values(by="budget", ascending=False).head(159)

,budget
acteur_name,
Robert Downey Jr.,1.522000e+09
Mark Ruffalo,1.147000e+09
Chris Evans,5.760000e+08
Chris Hemsworth,5.710000e+08
Tom Hanks,5.250000e+08
...,...
Tom Hardy,1.000000e+08
George Clooney,1.000000e+08
Fionn Whitehead,1.000000e+08


### Catégorisation

In [8]:
top_tier_actor_df = acteurs_df.groupby(by="acteur_name").sum()[['budget']].sort_values("budget", ascending=False)

In [9]:
top_tier_actor_df

,budget
acteur_name,
Mac,4.043730e+09
Johnny Depp,3.407500e+09
Robert Downey Jr.,3.175500e+09
Leon,2.895191e+09
Will Smith,2.653100e+09
...,...
Lalor Roddy,1.000000e+00
Helena Bereen,1.000000e+00
Daniel Furlan,9.082063e-01


In [10]:
def connupaconnu(X, ratio = 3):
    if (X['budget'] >= 100_000_000) & (X['ratio'] >= 3):
        return 1
    return 0

In [11]:
acteurs_df.sort_values('year', ascending=True, inplace= True)

In [12]:
acteurs_df['connu'] = acteurs_df.apply(connupaconnu, axis = 1)

In [13]:
acteurs_df['nbsucces'] = acteurs_df['connu']

In [14]:
new_df = acteurs_df

In [15]:
new_df['nbsuccess'] = acteurs_df.groupby(by  ='acteur_name')['connu'].cumsum(axis = 0)

In [16]:
new_df = new_df.sort_values('year', ascending=True)

In [17]:
new_df.drop(columns='nbsucces', inplace = True)

In [18]:
new_df['connu2'] = new_df['nbsuccess'].apply(lambda x : 1 if x >=1 else 0)

In [19]:
new_df.drop(columns="connu", inplace = True)

In [20]:
new_df.rename(columns={"connu2" : "connu"}, inplace=True)

In [21]:
new_df[new_df['acteur_name'].str.contains('Leonardo DiCaprio')]

,Unnamed: 0,acteur_name,title,year,month,budget,income,ratio,nbsuccess,connu
11077,11077,Leonardo DiCaprio,Buon compleanno Mr. Grape,1993,3,11000000.0,10032765,0.912070,0,0
11079,11079,Leonardo DiCaprio,Poeti dall'inferno,1995,8,8000000.0,340139,0.042517,0,0
11078,11078,Leonardo DiCaprio,Ritorno dal nulla,1995,4,2000000.0,2381087,1.190543,0,0
11080,11080,Leonardo DiCaprio,La stanza di Marvin,1996,4,23000000.0,12803305,0.556665,0,0
11081,11081,Leonardo DiCaprio,Romeo + Giulietta di William Shakespeare,1996,2,14500000.0,151842560,10.471901,0,0
11082,11082,Leonardo DiCaprio,Titanic,1997,1,200000000.0,2195169869,10.975849,1,1
11083,11083,Leonardo DiCaprio,La maschera di ferro,1998,3,35000000.0,182968902,5.227683,1,1
11084,11084,Leonardo DiCaprio,The Beach,2000,2,50000000.0,144056873,2.881137,1,1
11085,11085,Leonardo DiCaprio,Gangs of New York,2002,1,100000000.0,193772504,1.937725,1,1
11086,11086,Leonardo DiCaprio,Prova a prendermi,2002,1,52000000.0,352114312,6.771429,1,1


In [22]:
new_df

,Unnamed: 0,acteur_name,title,year,month,budget,income,ratio,nbsuccess,connu
0,0,Werner Krauss,Il gabinetto del dottor Caligari,1920,2,1.800000e+04,8811,0.489500,0,0
1,1,Conrad Veidt,Il gabinetto del dottor Caligari,1920,2,1.800000e+04,8811,0.489500,0,0
2,2,Friedrich Feher,Il gabinetto del dottor Caligari,1920,2,1.800000e+04,8811,0.489500,0,0
3,3,Amund Rydland,Markens grøde,1921,12,2.926274e+04,4272,0.145988,0,0
4,4,Karen Poulsen,Markens grøde,1921,12,2.926274e+04,4272,0.145988,0,0
...,...,...,...,...,...,...,...,...,...,...
32402,32402,Sabin Tambrea,Narziss und Goldmund,2020,3,1.157400e+07,233325,0.020159,0,0
27653,27653,Rodney Lam,Suriname,2020,2,2.314800e+06,544343,0.235158,0,0
27654,27654,Josylvio,Suriname,2020,2,2.314800e+06,544343,0.235158,0,0
26263,26263,Margot Robbie,Birds of Prey e la fantasmagorica rinascita di...,2020,2,8.450000e+07,201858461,2.388858,1,1


In [23]:
liste_fi = new_df[['acteur_name','title','year','nbsuccess','connu']].values.tolist()

In [24]:
new_df['totalsuccess'] = new_df.groupby(by = 'title').cumsum()['nbsuccess']

In [25]:
# new_df['totalsuccess2'] = new_df.groupby(by = 'title')['totalsuccess']


# df.groupby(['Mt'], sort=False)['count'].max()

In [26]:
total_success = pd.DataFrame(new_df.groupby(['title'], sort = False)['totalsuccess'].max())

In [30]:
total_success.reset_index(inplace=True)

In [31]:
total_success[total_success['title'].str.contains('Avengers')]

,title,totalsuccess
2790,The Avengers - Agenti speciali,0
8428,The Avengers,7
9700,Avengers: Age of Ultron,10
11206,Avengers: Infinity War,16
11542,Avengers: Endgame,19


In [32]:
total_success.sort_values('totalsuccess', ascending=False).head(50)

,title,totalsuccess
11542,Avengers: Endgame,19
11206,Avengers: Infinity War,16
11821,Fast & Furious - Hobbs & Shaw,14
11546,C'era una volta a... Hollywood,13
10926,Pirati dei Caraibi - La vendetta di Salazar,13
10561,Captain America: Civil War,13
9869,Revenant - Redivivo,13
10844,Guardiani della Galassia Vol. 2,12
10829,Fast & Furious 8,11
11822,Jumanji - The Next Level,11


# Chercher l'acteur qu'on veut (optionnel)

In [33]:
acteurs_df[acteurs_df['acteur_name'].str.contains("Leonardo DiCaprio")].sort_values('budget', ascending=False).sort_values('year')

,Unnamed: 0,acteur_name,title,year,month,budget,income,ratio,connu,nbsucces,nbsuccess
11077,11077,Leonardo DiCaprio,Buon compleanno Mr. Grape,1993,3,11000000.0,10032765,0.912070,0,0,0
11078,11078,Leonardo DiCaprio,Ritorno dal nulla,1995,4,2000000.0,2381087,1.190543,0,0,0
11079,11079,Leonardo DiCaprio,Poeti dall'inferno,1995,8,8000000.0,340139,0.042517,0,0,0
11081,11081,Leonardo DiCaprio,Romeo + Giulietta di William Shakespeare,1996,2,14500000.0,151842560,10.471901,0,0,0
11080,11080,Leonardo DiCaprio,La stanza di Marvin,1996,4,23000000.0,12803305,0.556665,0,0,0
11082,11082,Leonardo DiCaprio,Titanic,1997,1,200000000.0,2195169869,10.975849,1,1,1
11083,11083,Leonardo DiCaprio,La maschera di ferro,1998,3,35000000.0,182968902,5.227683,0,0,1
11084,11084,Leonardo DiCaprio,The Beach,2000,2,50000000.0,144056873,2.881137,0,0,1
11085,11085,Leonardo DiCaprio,Gangs of New York,2002,1,100000000.0,193772504,1.937725,0,0,1
11086,11086,Leonardo DiCaprio,Prova a prendermi,2002,1,52000000.0,352114312,6.771429,0,0,1


In [34]:
acteurs_df.drop(columns='nbsucces', inplace = True)

In [35]:
acteurs_df[acteurs_df['acteur_name'].str.contains("Leonardo DiCaprio")]
acteurs_df['shifted'] = acteurs_df['nbsuccess'] - acteurs_df['connu']

In [41]:
acteurs_df[['acteur_name','title','shifted']].groupby(by="title")

In [43]:
acteurs_df

,Unnamed: 0,acteur_name,title,year,month,budget,income,ratio,connu,nbsuccess,shifted
0,0,Werner Krauss,Il gabinetto del dottor Caligari,1920,2,1.800000e+04,8811,0.489500,0,0,0
1,1,Conrad Veidt,Il gabinetto del dottor Caligari,1920,2,1.800000e+04,8811,0.489500,0,0,0
2,2,Friedrich Feher,Il gabinetto del dottor Caligari,1920,2,1.800000e+04,8811,0.489500,0,0,0
3,3,Amund Rydland,Markens grøde,1921,12,2.926274e+04,4272,0.145988,0,0,0
4,4,Karen Poulsen,Markens grøde,1921,12,2.926274e+04,4272,0.145988,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
27115,27115,Souleymane Sylla,Tolo Tolo,2020,1,2.314800e+07,52186426,2.254468,0,0,0
27116,27116,Manda Touré,Tolo Tolo,2020,1,2.314800e+07,52186426,2.254468,0,0,0
35480,35480,Dev,Wendy,2020,4,6.000000e+06,144564,0.024094,0,0,0
33210,33210,Charlie Heaton,New Mutants,2020,9,6.700000e+07,10387310,0.155034,0,0,0


In [47]:
total_success = pd.DataFrame(acteurs_df.groupby(['title'], sort = False)[['acteur_name','shifted']].max())
total_success.reset_index(inplace = True)

In [49]:
total_success[total_success['acteur_name'].str.contains('Leonardo DiCaprio')]

,title,acteur_name,shifted
1824,Buon compleanno Mr. Grape,Leonardo DiCaprio,0
2310,Romeo + Giulietta di William Shakespeare,Leonardo DiCaprio,0
2532,Titanic,Leonardo DiCaprio,0
2663,La maschera di ferro,Leonardo DiCaprio,1
3736,Gangs of New York,Leonardo DiCaprio,1
4233,The Aviator,Leonardo DiCaprio,1
5112,Blood Diamond - Diamanti di sangue,Leonardo DiCaprio,1
6295,Revolutionary Road,Leonardo DiCaprio,1
7210,Inception,Leonardo DiCaprio,1
7785,J. Edgar,Leonardo DiCaprio,2


In [52]:
acteurs_df[acteurs_df['acteur_name'].str.contains('Margot Robbie')]

,Unnamed: 0,acteur_name,title,year,month,budget,income,ratio,connu,nbsuccess,shifted
26256,26256,Margot Robbie,The Wolf of Wall Street,2013,1,100000000.0,392000694,3.920007,1,1,0
26255,26255,Margot Robbie,Suite Francese,2014,3,17361000.0,9337930,0.537868,0,1,1
26258,26258,Margot Robbie,Focus - Niente è come sembra,2015,3,50100000.0,158762963,3.168921,0,1,1
26257,26257,Margot Robbie,Sopravvissuti,2015,8,7500000.0,381839,0.050912,0,1,1
26259,26259,Margot Robbie,Whiskey Tango Foxtrot,2016,5,35000000.0,24972139,0.713490,0,1,1
26260,26260,Margot Robbie,Tonya,2017,3,11000000.0,53939297,4.903572,0,1,1
26261,26261,Margot Robbie,Bombshell - La voce dello scandalo,2019,4,32000000.0,59863792,1.870743,0,1,1
26262,26262,Margot Robbie,C'era una volta a... Hollywood,2019,9,90000000.0,374343626,4.159374,0,1,1
26263,26263,Margot Robbie,Birds of Prey e la fantasmagorica rinascita di...,2020,2,84500000.0,201858461,2.388858,0,1,1


In [55]:
last_df = pd.DataFrame(acteurs_df.groupby('title').sum()["shifted"])

In [57]:
last_df.reset_index(inplace = True)

In [84]:
last_df[last_df['title'].str.contains('Avengers')]

,title,shifted
952,Avengers: Age of Ultron,7
953,Avengers: Endgame,16
954,Avengers: Infinity War,13
9968,The Avengers,4
9969,The Avengers - Agenti speciali,0


In [86]:
acteurs_df

,Unnamed: 0,acteur_name,title,year,month,budget,income,ratio,connu,nbsuccess,shifted
0,0,Werner Krauss,Il gabinetto del dottor Caligari,1920,2,1.800000e+04,8811,0.489500,0,0,0
1,1,Conrad Veidt,Il gabinetto del dottor Caligari,1920,2,1.800000e+04,8811,0.489500,0,0,0
2,2,Friedrich Feher,Il gabinetto del dottor Caligari,1920,2,1.800000e+04,8811,0.489500,0,0,0
3,3,Amund Rydland,Markens grøde,1921,12,2.926274e+04,4272,0.145988,0,0,0
4,4,Karen Poulsen,Markens grøde,1921,12,2.926274e+04,4272,0.145988,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
27115,27115,Souleymane Sylla,Tolo Tolo,2020,1,2.314800e+07,52186426,2.254468,0,0,0
27116,27116,Manda Touré,Tolo Tolo,2020,1,2.314800e+07,52186426,2.254468,0,0,0
35480,35480,Dev,Wendy,2020,4,6.000000e+06,144564,0.024094,0,0,0
33210,33210,Charlie Heaton,New Mutants,2020,9,6.700000e+07,10387310,0.155034,0,0,0


In [99]:
laaast = pd.DataFrame(last_df.merge(acteurs_df, on='title').drop(columns=["connu","nbsuccess","shifted_y","Unnamed: 0"])).drop_duplicates()

In [100]:
# last_df.to_csv('../raw_data/cat_acteur.csv')

In [101]:
laaast[laaast['title'].str.contains('Avengers: Endgame')]

,title,shifted_x,acteur_name,year,month,budget,income,ratio
2959,Avengers: Endgame,16,Mark Ruffalo,2019,4,356000000.0,2797800564,7.85899
2960,Avengers: Endgame,16,Chris Evans,2019,4,356000000.0,2797800564,7.85899
2961,Avengers: Endgame,16,Robert Downey Jr.,2019,4,356000000.0,2797800564,7.85899


In [102]:
laaast.rename(columns={"shifted_x":'shifted'}, inplace=True)

In [103]:
laaast.to_csv('../raw_data/cat_acteur.csv')